In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,8)

# # 青海省部分地区
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','天津市'))
# roi = ee.Geometry.Rectangle([89.8565, 34.6503,91.4664, 35.8282])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

def maskL8sr_reverse(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask.remap([0,1],[1,0]))

# 图像可视化参数
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})

# RF训练

In [5]:
## 导入采集的样本集
points_collection = ee.FeatureCollection('users/311605001111/tianjing_2018')
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))
a = points_collection.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())


# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [117.52114219432853, 39.65410342583302]}, 'id': '000000000000000004c8', 'properties': {'AWEI': -3582.75, 'B2': 657, 'B3': 1063, 'B4': 1084, 'B5': 2405, 'B6': 1965, 'B7': 1369, 'Image_id': 'LC08_122033_20180417', 'NDVI': 0.378618515333907, 'mNDWI': -0.297886393659181, 'ndvi_mndwi': 0.676504908993087, 'waterclass': 0}}
total sample number:3244
水体样本的数目： 784


In [6]:
# 'LC08_122033_20180111'
# 'LC08_123033_20180203'
# 'LC08_122033_20180316'
# 'LC08_123033_20180408'
# 'LC08_122033_20180503'
# 'LC08_122033_20180620'
# 'LC08_122033_20180706' 
# 'LC08_122033_20180823'
# 'LC08_122033_20180924'
# 'LC08_122033_20181026'
# 'LC08_123033_20181001'
# 'LC08_123032_20181118'
image_id = 'LC08_123033_20181001'
id = 'LANDSAT/LC08/C01/T1_SR/' + image_id
image2 = water_index(ee.Image(id))

Map.addLayer(image2,visParams,'image2')
Map.addLayer(maskL8sr(image2),visParams,'mask image2')

# 校正

In [7]:
# img指经过指数计算，但未做云掩膜的图像
def AutomaticCorrection(img):
    image = maskL8sr(img).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    invalidPixel = maskL8sr_reverse(img).select('B2').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image.remap([0,1],[1,2]).rename('waterclass').float()]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': img.geometry(),
        'scale': 30,
        'bestEffort': True
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    basemap = img.select('B2').gt(0).remap([0,1],[1,0]).rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water.set({'occurrence_threshold':occurrence_threshold})

In [8]:
a = AutomaticCorrection(image2)
print(a.get('occurrence_threshold').getInfo())
Map.addLayer(a.selfMask(),{'palette':['red']},"a")

5


# 验证

In [9]:
img = image2
image = maskL8sr(img).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
Map.addLayer(image.selfMask(),{'palette':['blue']},"water")

In [ ]:
### 人工选取的样本点
# 要求：水体类别属性为 label; 影像属性为 Image_id 。
man_point = ee.FeatureCollection('users/311605001111/tianjin_2018').filter(ee.Filter.eq('Image_id',image_id))
print(man_point.size().getInfo())

water_image = ee.Image.constant(1).toFloat().rename('label')
land_image = ee.Image.constant(0).toFloat().rename('label')
water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
image_waterclass = ee.ImageCollection([water,land]).sum()
# Map.addLayer(image_waterclass,{'palette':['green','blue'],'min':0,'max':1},"image_waterclass")

In [ ]:
# 采样
validation_points = image_waterclass.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 300000,
    'seed': 0,
    'geometries': True,
})

print(validation_points.size().getInfo())
print(validation_points.first().getInfo())

In [ ]:
# 采样
inter_points = image.sampleRegions(**{
    'collection': validation_points,
    'properties': ['label'],
    'scale': 30,
    'geometries': True,
})

print(inter_points.size().getInfo())
print(inter_points.first().getInfo())

In [ ]:
# 利用误差矩阵进行验证
test_accuracy = inter_points.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
# Map.addLayer(land_random,{'color': 'red', 'pointSize': 100,'fillColor': 'red'},'points');

# 制图

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
%pylab inline
region = [89.8565, 34.6503,91.4664, 35.8282] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'waterbody frequency(Auto-Correction/validPixel/2018)', fontsize=20)

In [ ]:
# permanent waterbody
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8565, 34.6503,91.4664, 35.8282] 
vis = {'palette':['D3D3D3','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'permanent water(Auto-Correction/validPixel/2018)', fontsize=20)